In [1]:
import numpy as np
import torch
import json
import os

from libraries.dataset        import standardize_dataset, check_extend_POSCAR
from libraries.graph          import graph_POSCAR_encoding
from libraries.structure      import compute_diffraction_pattern
from torch_geometric.data     import Data
from pymatgen.core            import Structure

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# In case database is created from scratch (otherwise, it is not being used)
data_path = '../MP/Loaded_PT'

# Define diffraction type ('neutron', 'xrd' or 'EPA')
target = 'EPA'

# Define folder in which all data will be stored
data_folder = f'data/GM_PT_EPA-voronoi'

# Define name for storing dataset basic description
dataset_parameters_name = f'{data_folder}/dataset_parameters.json'

encoding_type      = 'voronoi'  # 'voronoi' or 'sphere-images'
distance_threshold = 6  # Used in general

minimum_lattice_vector = 3 * distance_threshold  # Allowing three convolutions

# Define basic dataset parameters for tracking data
dataset_parameters = {
    'input_folder': data_path,
    'output_folder': data_folder,
    'target': target,
    'encoding_type': encoding_type,
    'distance_threshold': distance_threshold,
    'minimum_lattice_vector': minimum_lattice_vector
}

if not os.path.exists(data_folder):
    os.system(f'mkdir {data_folder}')

# Dump the dictionary with numpy arrays to a JSON file
with open(dataset_parameters_name, 'w') as json_file:
    json.dump(dataset_parameters, json_file)

# Generation of graph database for training

Load the datasets, already standardized if possible.

In [3]:
# Generate the raw dataset from scratch, and standardize it

# Read all materials within the database
materials = os.listdir(data_path)

dataset = []
labels  = []
for material in materials[:1000]:
    try:
        # Try to read the polymorphs
        polymorphs = os.listdir(f'{data_path}/{material}')
    except:
        continue
    
    print(material)
    for polymorf in polymorphs:
        # Path to folder containing the POSCAR
        path_to_POSCAR = f'{data_path}/{material}/{polymorf}'
        
        # Check that the folder is valid
        if os.path.exists(path_to_POSCAR):
            print(f'\t{polymorf}')
            
            try:
                # Load pymatgen structure object
                structure = Structure.from_file(f'{path_to_POSCAR}/POSCAR')
                
                # Check that POSCAR is big enough, otherwise extend it where necessary
                structure = check_extend_POSCAR(structure, minimum_lattice_vector)
                
                nodes, edges, attributes = graph_POSCAR_encoding(structure,
                                                                 encoding_type=encoding_type,
                                                                 distance_threshold=distance_threshold)
            except:
                print(f'Error: {material} {polymorf} not loaded')
                continue
            
            if target == 'EPA':
                # Load ground state energy per atom
                extracted_target = [float(np.loadtxt(f'{path_to_POSCAR}/EPA'))]
            elif (target == 'neutron') or (target == 'xrd'):
                # Compute diffraction pattern from given structure
                extracted_target = compute_diffraction_pattern(structure, diffraction=target)
            else:
                # Do not extract anything
                extracted_target = [0]
            
            # Construct temporal graph structure
            graph = Data(x=nodes,
                         edge_index=edges.t().contiguous(),
                         edge_attr=attributes.ravel(),
                         y=torch.tensor(extracted_target, dtype=torch.float)
                        )

            # Append to dataset and labels
            dataset.append(graph)
            labels.append(f'{material}-{polymorf}')

CeSnRh
	P-62m
LuSi
	Cmcm
	P-6m2
Gd2Ti2O7
	Fd-3m
	C2-m
LiAs3H2O9
	P2_1-c
VAg2HgO4
	I-42d
Y5Be6Fe3(SiO5)6
	P1
CuCN2
	Cmcm
BaSrEuCrO6
	F-43m
Ta2N3
	Pnma
Nd6Al43W4
	P6_3-mcm
CdSO4
	Cmcm
	Pmn2_1
	P3m1
	C2-m
Mg6ZnFe
	Amm2
BaCeEuSbO6
	P-1
	F-43m
	P222
Li2Mn2CoO6
	P-1
	Cmce
Na(CrS2)2
	P-1
	I4_1-amd
	Pmmn
	Fd-3m
	Pca2_1
	Pnma
	P2-m
	C2-m
KCu2BiS3
	P-1
SrMg14Sn
	P-6m2
	Amm2
LiZnInF6
	P321
KCe(PO3)4
	P2_1-c
	P2_1
K2NiAs2
	Cmcm
Nb2SiNi3
	P6_3-mmc
CaCoF4
	C2-c
Ta2Co3Si
	P6_3-mmc
LaEr4Se7
	Cm
NdTiFe11N
	Imm2
YTaP2SO12
	Cc
MgAl3SiBO9
	Pnma
CaZn2(H5O4)2
	P2_1-c
K3CeI6
	Fm-3m
Mn3Ni2P6WO24
	R3
H6PbC2S2O7
	P-1
BaFe4O7
	P6_3mc
	P6_3-m
Nb2CoS4
	P-3m1
	Pnma
W2Br5
	P2_1-c
ZrP
	P6_3-mmc
	Fm-3m
BaClF
	P4-nmm
NaPbO3
	Pm-3m
KAl4(SiO6)2
	C2-c
Y4Cu7O16
	P-1
Li5Mn5Cr2O12
	C2
U11Se20
	.DS_Store
Error: U11Se20 .DS_Store not loaded
	P4nc
	P4
Yb2MgTi2O6
	R3
	R-3
MgTiMn3O8
	Cm
	P1
	C2-m
Na6Y4Al4Si3(S3O16)3
	P1
LiNi7O7F
	Pm
Na4Ga4Si19
	P1
BaMgFeF7
	P2_1-c
Sn4Te3Se
	R3m
Sr2Cd2Cu(SO)2
	I4-mmm
Na14Ca4Hf4Si7S5O48
	P1
PrTaTiO

In [4]:
# Standardize dataset
dataset_std, dataset_parameters = standardize_dataset(dataset, transformation='inverse-quadratic')

# Save dataset

In [5]:
labels_name                 = f'{data_folder}/labels.pt'
dataset_name                = f'{data_folder}/dataset.pt'
dataset_name_std            = f'{data_folder}/standardized_dataset.pt'
dataset_parameters_name_std = f'{data_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions

torch.save(labels,      labels_name)
torch.save(dataset,     dataset_name)
torch.save(dataset_std, dataset_name_std)

# Convert torch tensors to numpy arrays
numpy_dict = {}
for key, value in dataset_parameters.items():
    try:
        numpy_dict[key] = value.cpu().numpy().tolist()
    except:
        numpy_dict[key] = value

# Dump the dictionary with numpy arrays to a JSON file
with open(dataset_parameters_name_std, 'w') as json_file:
    json.dump(numpy_dict, json_file)